In [18]:
%matplotlib qt
import os
import math

import numpy as np
import cv2
import matplotlib.pyplot as plt


In [19]:
dataPath = "data/videos/2016_0718_200947_002"  # input video path
cornerDataPath = "data/corners.npy"
calibrationDataPath = "data/calibration.xml"
readExistingCornerData = False

patternSize = (8, 6)  # number of corners in the calibration pattern (rows, columns)
subPixWinSize = (5, 5)  # depends on the resolution, 5, 5 seems ok for this camera resolution
maxNumSamples = 40  # if there are more than this collected samples, all samples are shuffled and subsampled
skipFrames = 120 # how many frames do we skip while seeking video for calibration shots
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# setting up object points
patternSquareSideLength = 1.6
objPoints = np.zeros((patternSize[0]*patternSize[1], 3), np.float32)
objPoints[:, :2] = np.mgrid[0:patternSize[0], 0:patternSize[1]].T.reshape(-1, 2).astype(np.float32)
objPoints *= patternSquareSideLength

In [31]:
# read sample video for out frame size
framePaths = list(map(lambda f: os.path.join(dataPath, f), filter(lambda f: ".png" in f, os.listdir(dataPath))))

frames = list(map(lambda f: cv2.cvtColor(cv2.imread(f), cv2.COLOR_BGR2GRAY), framePaths))

print("Read {} frames".format(len(frames)))

Read 137 frames


In [35]:
validFrames = []

allCorners = []  # collection of all corners for calibration purposes
if not readExistingCornerData:

    # Enable interactive mode
    plt.ion()
    fig = plt.figure()

    for frame in frames:
        quitReading = False

        # try detecting the chart
        patternWasFound, corners = cv2.findChessboardCorners(frame, patternSize, cv2.CALIB_CB_FAST_CHECK)

        if patternWasFound:
            print("Found corners in frame {}".format(len(allCorners)))
            corners = cv2.cornerSubPix(frame, corners, subPixWinSize, (-1, -1), criteria)
            allCorners.append(corners)
            frame_bgr = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
            cv2.drawChessboardCorners(frame_bgr, patternSize, corners, patternWasFound)

            frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            validFrames.append(frame)

            plt.imshow(frame_rgb)
            plt.show()
            plt.pause(0.001)

    allCorners = np.array(allCorners)
    np.save(cornerDataPath, allCorners)

    print("Saved corners to {}".format(cornerDataPath))
    plt.ioff()
else:
    allCorners = np.load(cornerDataPath)

numSamples = allCorners.shape[0]
print("Collected {} corner sampels.".format(numSamples))

Found corners in frame 0
Found corners in frame 1
Found corners in frame 2
Found corners in frame 3
Found corners in frame 4
Found corners in frame 5
Found corners in frame 6
Found corners in frame 7
Found corners in frame 8
Found corners in frame 9
Found corners in frame 10
Found corners in frame 11
Found corners in frame 12
Found corners in frame 13
Found corners in frame 14
Found corners in frame 15
Found corners in frame 16
Found corners in frame 17
Found corners in frame 18
Found corners in frame 19
Found corners in frame 20
Found corners in frame 21
Found corners in frame 22
Found corners in frame 23
Found corners in frame 24
Found corners in frame 25
Found corners in frame 26
Saved corners to data/corners.npy
Collected 27 corner sampels.


In [5]:
# take just number of samples
if numSamples > maxNumSamples:
    # np.random.shuffle(sampleIds)

    ratio = int(math.ceil(numSamples / maxNumSamples))
    print("ratio: {}".format(ratio))

    sampleIds = np.arange(0, numSamples, ratio)

    allCorners = allCorners[sampleIds]
    numSamples = allCorners.shape[0]

    print("Num samples trimmed to {}, according to selection:\n{}".format(numSamples, sampleIds))


ratio: 4
Num samples trimmed to 38, according to selection:
[  0   4   8  12  16  20  24  28  32  36  40  44  48  52  56  60  64  68
  72  76  80  84  88  92  96 100 104 108 112 116 120 124 128 132 136 140
 144 148]


In [6]:
# now let's try to calibrate
objPoints = [objPoints] * numSamples
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objPoints, allCorners, frameSize, None, None)

if ret:
    print("Matrix:\n{}".format(mtx))
    print("Distortion:\n{}".format(dist))

    storage = cv2.FileStorage(calibrationDataPath, cv2.FILE_STORAGE_WRITE)
    storage.write("mtx", mtx)
    storage.write("dist", dist)
    storage.release()

    # print(rvecs)
    # print(tvecs)
else:
    print("Calibration failed")
    sys.exit(1)

# find optimal camera matrix
h, w = frame.shape[0:2]


Matrix:
[[4.11548576e+03 0.00000000e+00 3.25537652e+02]
 [0.00000000e+00 4.74418170e+03 5.76401937e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion:
[[-5.07988557e+00 -3.86020276e+02 -4.57061429e-01  6.81333564e-01
   6.76501215e+03]]


1280 720


In [12]:
R, _ = cv2.Rodrigues(rvecs[1])
np.matmul(R, objPoints[1].T)

array([[ 0.        ,  1.57960606,  3.15921212,  4.7388183 ,  6.31842424,
         7.89803018,  9.47763659, 11.05724207,  0.08096428,  1.66057034,
         3.2401764 ,  4.81978258,  6.39938852,  7.97899446,  9.55860088,
        11.13820635,  0.16192857,  1.74153463,  3.32114069,  4.90074686,
         6.48035281,  8.05995875,  9.63956516, 11.21917063,  0.24289286,
         1.82249892,  3.40210497,  4.98171115,  6.56131709,  8.14092304,
         9.72052945, 11.30013492,  0.32385713,  1.90346319,  3.48306925,
         5.06267543,  6.64228137,  8.22188731,  9.80149373, 11.3810992 ,
         0.40482141,  1.98442747,  3.56403353,  5.14363971,  6.72324565,
         8.30285159,  9.882458  , 11.46206348],
       [ 0.        ,  0.02372559,  0.04745118,  0.07117677,  0.09490236,
         0.11862795,  0.14235355,  0.16607913,  1.46358113,  1.48730672,
         1.51103231,  1.5347579 ,  1.55848349,  1.58220908,  1.60593467,
         1.62966025,  2.92716225,  2.95088784,  2.97461343,  2.99833903,
   

In [14]:
#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')

projections = []
for i in range(len(rvecs)):
    R, _ = cv2.Rodrigues(rvecs[i])
    Z = np.matmul(R, objPoints[i].T) + np.array(tvecs[i])
    projections.append(Z)

    #ax.scatter(Z[0], Z[1], Z[2])

#ax.set_xlabel('X')
#ax.set_ylabel('Y')
#ax.set_zlabel('Z')

#plt.show()

In [40]:
import csv

with open('data/projections.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for table in project
    ions[0]:
        for row in table.T:
            print(row)
            print()
            #writer.writerow(row.tolist())

3.2793197023161427

4.8570014554016225

6.434683208487103

8.01236507911903

9.590046714658062

11.167728350197093

12.745410455921919

14.323091621275156

3.2307198473979937

4.8084016004834735

6.386083353568953

7.963765224200882

9.541446859739914

11.119128495278945

12.696810601003769

14.274491766357006

3.1821199924798442

4.7598017455653245

6.337483498650804

7.915165369282732

9.492847004821764

11.070528640360795

12.64821074608562

14.225891911438858

3.1335201339407237

4.711201887026204

6.288883640111683

7.866565510743612

9.444247146282644

11.021928781821675

12.5996108875465

14.177292052899737

3.084920282643546

4.662602035729026

6.240283788814505

7.817965659446434

9.395647294985466

10.973328930524497

12.551011036249323

14.12869220160256

3.036320431346368

4.614002184431848

6.191683937517327

7.7693658081492565

9.347047443688288

10.924729079227319

12.502411184952145

14.080092350305382

-9.594674552713215

-9.442410238793988

-9.29014592487476

-9.13788

In [38]:
projections[0].T[0]

array([  3.2793197 ,  -9.59467455, 117.52594889])